# Install Python Libraries

In [2]:
!pip install "pymssql<3.0"
!pip install tensorflow
!pip install keras
!pip install pandas-profiling

     |████████████████████████████████| 1.4MB 3.5MB/s 


In [0]:
import pandas as pd
import numpy as np
import pandas as pd
import time
import gc
import matplotlib.pyplot as plt
import pickle
import pandas_profiling
import pymssql
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,r2_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import StandardScaler, normalize, OneHotEncoder,LabelEncoder,Normalizer
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Loading Data from Sql Server


In [9]:
#Connect to Database
conn = pymssql.connect(server="zillowdata.cnyrvm7s6vwa.us-east-1.rds.amazonaws.com", user="user_readonly",password="Zillow",database="Zillow")
sql = r"SELECT * FROM [Zillow].[dbo].[zillow_data_input] where home_type in ('Condominium','SingleFamily','Townhouse')"
df = pd.read_sql(sql, conn)
conn.close()
#print(df["zestimate_amount"])
#df["zestimate_amount"] = np.log(df["zestimate_amount"].astype(float))
df = df.loc[~df.state_name.isna(),]
le = LabelEncoder()
df["home_text"] = df["home_type"]
df["home_type"] = df[["home_type"]].apply(le.fit_transform)["home_type"]

df["location"] = df[["state_name"]].apply(le.fit_transform)["state_name"]
use_features = ["bedrooms","bathrooms","tax_year","tax_value","property_size","home_size","year_built","location","zestimate_amount"]
print("Data loaded")

Data loaded


#Pandas Profiling Report

In [5]:
import pandas_profiling

report = pandas_profiling.ProfileReport(df)
display(report)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,22
Number of observations,299184
Total Missing (%),1.7%
Total size in memory,50.2 MiB
Average record size in memory,176.0 B
Numeric,12
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [0]:


df = df[use_features].dropna()
transformer = Normalizer().fit(df[use_features].values) 
df = pd.DataFrame(transformer.transform(df[use_features].values), columns = use_features)

#Pre-processing

In [0]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
train,test = train_test_split(df[['bedrooms','property_size']],test_size = 0.3)
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(train)  
IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=np.NAN, n_nearest_features=None,
                 random_state=0, sample_posterior=False, tol=0.001,
                 verbose=0)
imp.transform(test)

array([[3.41080314e-06, 1.75946280e-02],
       [8.20653155e-06, 2.62609010e-02],
       [6.05421052e-06, 1.09883921e-02],
       ...,
       [1.01084833e-05, 1.87133297e-02],
       [6.23303066e-06, 2.69266925e-03],
       [4.27097118e-06, 1.30229029e-01]])

In [0]:
df = df.dropna()
#df['year_built'] = df['year_built'].astype(int)
#df = df.loc[df["state_name"]=="Washington",] 
train,test = train_test_split(df,test_size = 0.3)

#train = train.loc[train["home_type"] == "SingleFamily",]
train, test = train_test_split(df, test_size=0.3)
train,test = train.dropna(),test.dropna()
use_features = ["bedrooms","bathrooms","tax_year","tax_value","property_size","home_size","year_built","location","home_type"]

#train["zestimate_amount"] = np.log(df["zestimate_amount"].astype(float))

y_train = train.pop('zestimate_amount').values
x_train = train[use_features]
y_test = test.pop('zestimate_amount').values
x_test = test[use_features]

#Linear Regression


In [11]:
regr = LinearRegression()
regr.fit(x_train, y_train)

y_pred = regr.predict(x_train)
y_pred2 = regr.predict(x_test[use_features])
print(y_pred.shape,y_train.shape)
print(mean_squared_error(y_train,y_pred))
print("r^2 in sample",r2_score(y_train,y_pred))
print("r^2 out of sample",r2_score(y_test,y_pred2))

#print(y_pred2)
#print(y_test)
#filename = 'linear_model.sav'
#pickle.dump(model, open(filename, 'wb'))

(163956,) (163956,)
375780808364.6029
r^2 in sample 0.5729654297767727
r^2 out of sample -1.545125447032928


# Random Forest

In [12]:
clf = RandomForestRegressor(n_estimators=25, max_depth=20, random_state=0)
clf.fit(x_train,y_train)
#print(list(x_train.columns))

#print(clf.feature_importances_)
importance = pd.DataFrame({"Variable":list(x_train.columns),"Value":clf.feature_importances_})

y_pred = clf.predict(x_train)
y_pred2 = clf.predict(x_test)
#print(mean_squared_error(y_train,y_pred))
print(r2_score(y_train,y_pred))

#print(mean_squared_error(y_test,y_pred2))
print(r2_score(y_test,y_pred2))

'''parameters = {"n_estimators":[20,60],"max_depth":[10,60]}
model = RandomForestRegressor(n_estimators=20, max_depth=2, random_state=0)
clf = GridSearchCV(model, parameters, cv=5)
clf.fit(x_train,y_train)

y_pred = clf.predict(x_train)
y_pred2 = clf.predict(x_test)
print(r2_score(y_train,y_pred))
print(r2_score(y_test,y_pred2))'''
                            


'''plt.bar(importance.Variable, importance.Value)
plt.title("Relative Feature Strength/Importance")
plt.xticks(rotation=90)
plt.show()'''

0.9625759315281195
0.8165111645590798


'plt.bar(importance.Variable, importance.Value)\nplt.title("Relative Feature Strength/Importance")\nplt.xticks(rotation=90)\nplt.show()'

In [0]:
one = x_train.loc[0:2]

clf.predict(one)
print(clf.predict(one))
#clf.predict(one)

[ 812793.43070734  536992.28398042 1289495.67372807 ...  359320.44408557
  797261.23736192  695891.34306975]


#Neural Network

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor


def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(use_features)]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

model = build_model()
model.summary()

'''y_pred = model.predict(x_train)
      y_pred2 = model.predict(x_test)
      print("r^2 in sample",r2_score(y_train,y_pred))
      print("r^2 out of sample",r2_score(y_test,y_pred2))'''

class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0:print('')

    print('.', end='')

EPOCHS = 1000

history = model.fit(
  x_train, y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


In [0]:
#model.fit(x_train,y_train)

y_pred = model.predict(x_train)
y_pred2 = model.predict(x_test)


print("r^2 in sample",r2_score(y_train,y_pred))
print("r^2 out of sample",r2_score(y_test,y_pred2))